In [15]:
import random

import numpy
from deap import base, creator, tools, algorithms

# Define the problem as a multi-objective optimization problem
creator.create("FitnessMulti", base.Fitness, weights=[1.0, -1.0])
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Define the problem-specific parameters
n = 5  # Number of physical machines
m = 10  # Number of virtual machines
s = 20 # Number of services
alpha = 0.5
beta = 0.5
num_generations = 50

# Define the bounds for decision variables (placement of services)
bounds = [(0, n - 1)] * m

# Define parameters. This will be loaded from the database
R = [[random.uniform(0, 1) for _ in range(m)] for _ in range(n)] # Request network delay of the machine
I = [[random.uniform(0, 1) for _ in range(m)] for _ in range(n)] # IPS currently executed
X = [[random.uniform(0, 1) for _ in range(m)] for _ in range(n)] # Maximum IPS possible
S = [[random.uniform(0, 1) for _ in range(m)] for _ in range(n)] # Response network delay of the machine

PI = [[random.uniform(0, 1) for _ in range(m)] for _ in range(n)] # Power consumption in idle
PM = [[random.uniform(0, 1) for _ in range(m)] for _ in range(n)] # Maximum power consumption

E = [[PI[i][j] + (PM[i][j] - PI[i][j])(I[i][j]/X[i][j]) for i in range(m)] for j in range(n)]

# Define the objective functions
def latency(individual):
    return [sum(R[i][j] + I[i][j] / X[i][j] + S[i][j] for i in range(n) for j in range(m))]

def energy_consumption(individual):
    return [sum(E[i][j] for i in range(n) for j in range(m))]

toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, n - 1)
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_int,), n=m)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Convert the fitness values to lists explicitly
def evaluate_individual(ind):
    lat, energy = latency(ind), energy_consumption(ind)
    return numpy.array([lat, energy])

toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=n - 1, indpb=0.2)
toolbox.register("select", tools.selBest)

if __name__ == "__main__":
    # Create the initial population
    population = toolbox.population(n=100)

    # Create statistics object to track performance
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min_lat", min)
    stats.register("min_energy", min)

    # Create a logbook to log statistics
    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "min_lat", "min_energy"

    # Run the optimization using a basic Genetic Algorithm (GA)
    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations, stats=stats,
                        halloffame=None, verbose=True)

    # Extract the Pareto front solutions (not necessarily Pareto-optimal in a basic GA)
    pareto_front = population

    # Print Pareto front solutions
    print("Pareto Front Solutions:")
    for ind in pareto_front:
        print("Latency:", ind.fitness.values[0][0])  # Extract the first element
        print("Energy Consumption:", ind.fitness.values[1][0])  # Extract the first element
        print("Placement:", ind)

gen	nevals	min_lat                                      	min_energy                                   
0  	100   	(array([123.42780134]), array([27.23021338]))	(array([123.42780134]), array([27.23021338]))
1  	74    	(array([123.42780134]), array([27.23021338]))	(array([123.42780134]), array([27.23021338]))
2  	76    	(array([123.42780134]), array([27.23021338]))	(array([123.42780134]), array([27.23021338]))
3  	77    	(array([123.42780134]), array([27.23021338]))	(array([123.42780134]), array([27.23021338]))
4  	82    	(array([123.42780134]), array([27.23021338]))	(array([123.42780134]), array([27.23021338]))
5  	80    	(array([123.42780134]), array([27.23021338]))	(array([123.42780134]), array([27.23021338]))
6  	67    	(array([123.42780134]), array([27.23021338]))	(array([123.42780134]), array([27.23021338]))
7  	77    	(array([123.42780134]), array([27.23021338]))	(array([123.42780134]), array([27.23021338]))
8  	74    	(array([123.42780134]), array([27.23021338]))	(array([123.4278